<a href="https://colab.research.google.com/github/sandnaja/ChatbotProject/blob/main/Train_QA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [3]:
import torch
torch.cuda.is_available()

False

In [ ]:
!pip -q install transformers

     |████████████████████████████████| 4.0 MB 5.0 MB/s 
     |████████████████████████████████| 596 kB 54.4 MB/s 
     |████████████████████████████████| 77 kB 6.1 MB/s 
     |████████████████████████████████| 880 kB 54.5 MB/s 
     |████████████████████████████████| 6.6 MB 44.6 MB/s 


In [ ]:
!pip -q install datasets

In [ ]:
from datasets import load_dataset
thaiqa_ds = load_dataset('thaiqa_squad')

In [ ]:
thaiqa_ds

In [ ]:
import pandas as pd
qa_train = pd.DataFrame(thaiqa_ds["train"])

In [ ]:
qa_train

In [ ]:
qa_train["context"]

In [ ]:
qa_train["context"][0]

In [ ]:
from lxml import etree
parser = etree.XMLParser(recover=True)
tree = etree.fromstring(qa_train["context"][0], parser=parser)
print(tree.tag)
print(tree.attrib)
print(tree.text)

In [ ]:
def extract_text(body):
  tree = etree.fromstring(body, parser=parser)
  return tree.text

In [ ]:
extract_text(qa_train["context"][0])

In [ ]:
qa_train["context_text"] = qa_train["context"].apply(extract_text)

In [ ]:
qa_train.head()

In [ ]:
!pip -q install pythainlp

In [ ]:
from pythainlp.tokenize import word_tokenize
qa_train["context_text_wordsegged"] = qa_train["context_text"].apply(lambda x: ' '.join(word_tokenize(x)))

In [ ]:
qa_train["question_wordsegged"] = qa_train["question"].apply(lambda x: ' '.join(word_tokenize(x)))

In [ ]:
qa_train.head()

In [ ]:
qa_train["answers_wordsegged"] =  qa_train["answers"].apply(lambda x: ' '.join(word_tokenize(x["answer"][0])))

In [ ]:
qa_train.head()

In [ ]:
answer_starts = []
for idx, row in qa_train.iterrows():
  try:
    answer_start = row["context_text_wordsegged"].index(row["answers_wordsegged"])
  except:
    answer_start = -1
  answer_starts.append(answer_start)

In [ ]:
qa_train["answer_starts"] = answer_starts

In [ ]:
qa_train.tail()

In [ ]:
qa_train.head()

In [ ]:
train_data = []
for idx, row in qa_train.iterrows():
  doc = {}
  doc["context"] = row["context_text_wordsegged"]
  doc["qas"] = [{"id":idx, "is_impossible":row["answer_starts"]<0,"question":row["question_wordsegged"],"answers":[{"text":row["answers_wordsegged"], "answer_start":row["answer_starts"]}]}]
  train_data.append(doc)

In [ ]:
train_data

In [ ]:
from transformers import AutoTokenizer, AutoModel
from google.colab import widgets

In [ ]:
pretrained = "monsoon-nlp/bert-base-thai"

In [ ]:
name = pretrained

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(name, use_fast=True)
model = AutoModel.from_pretrained(name)

In [ ]:
from transformers import pipeline

In [ ]:
!pip install matplotlib-venn

In [ ]:
!apt-get -qq install -y libfluidsynth1

In [ ]:
!apt-get -qq install -y libarchive-dev && pip install -U libarchive
import libarchive

In [ ]:
!apt-get -qq install -y graphviz && pip install pydot
import pydot

In [ ]:
!pip install cartopy
import cartopy

In [ ]:
!pip install simpletransformers

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from simpletransformers.question_answering import QuestionAnsweringModel,QuestionAnsweringArgs

In [ ]:
from simpletransformers.question_answering import QuestionAnsweringModel,QuestionAnsweringArgs


# Configure the model
model_args = QuestionAnsweringArgs()
model_args.train_batch_size = 32
model_args.evaluate_during_training = True
model_args.num_train_epochs = 3
model_args.overwrite_output_dir = True

# model_args = {
#     "reprocess_input_data": True,
#     'train_batch_size':32,
#     "use_early_stopping": True,
#     "early_stopping_delta": 0.01,
#     "early_stopping_metric": "mcc",
#     "early_stopping_metric_minimize": False,
#     "early_stopping_patience": 5,
#     "evaluate_during_training_steps": 500,
#     "fp16": False,
#     "num_train_epochs":3,
#     "overwrite_output_dir":True
# }

model = QuestionAnsweringModel(
    "bert", name, args=model_args
)

# Train the model
model.train_model(train_data, eval_data=train_data)

# Evaluate the model
result, texts = model.eval_model(train_data)

In [ ]:
QuestionAnsweringModel.save_model("./model-QA-wangchanberta")